# Import selenium và webdriver

* selenium - webdriver
* Beautiful soup
* time
* webdriver-manager để auto xử lý chrome driver
 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from webdriver_manager.chrome import ChromeDriverManager 

import time
import os

from bs4 import BeautifulSoup

# Đăng nhập và cấu hình driver

In [ ]:
SCREEN_WIDTH = 1920
SCREEN_HEIGHT = 1080
BATCH_SIZE = 25

def setup_driver():
    chrome_options = Options()
    # Thêm đường dẫn tới Chrome browser
    chrome_options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Đường dẫn mặc định của Chrome
    
    chrome_options.add_argument("--app")
    chrome_options.add_argument(f"--window-size={SCREEN_WIDTH},{SCREEN_HEIGHT}")
    chrome_options.add_argument("--window-position=0,0")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Đăng nhập vào web

* account: dihidromonooxit01012000@gmail.com
* password: 1!Aaaaaaaaaa

In [3]:
def sign_in(driver, email, password):
    driver.get("https://itviec.com/sign_in")

    time.sleep(2)

    email_input = driver.find_element(By.NAME, "user[email]")
    password_input = driver.find_element(By.NAME, "user[password]")

    email_input.send_keys(email)
    password_input.send_keys(password)

    submit_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    submit_button.click()

    time.sleep(5)
    print("Đăng nhập thành công!")

# Test cào thử cái gì đó


In [4]:
def test_crawl(driver):
    driver.get("https://itviec.com/it-jobs")
    time.sleep(5)
    test = driver.page_source
    print("Truy cập trang chủ ITviec thành công!")
    print(test[:500])  # In ra 500 ký tự đầu tiên của trang để kiểm tra

In [5]:
driver =setup_driver()
sign_in(driver, "dihidromonooxit01012000@gmail.com", "1!Aaaaaaaaaa")
test_crawl(driver)

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00CDA813+48355]
	(No symbol) [0x00C6C4B1]
	(No symbol) [0x00B75358]
	(No symbol) [0x00B91A9E]
	(No symbol) [0x00B90579]
	(No symbol) [0x00BC0C55]
	(No symbol) [0x00BC093C]
	(No symbol) [0x00BBA536]
	(No symbol) [0x00B982DC]
	(No symbol) [0x00B993DD]
	GetHandleVerifier [0x00F3AABD+2539405]
	GetHandleVerifier [0x00F7A78F+2800735]
	GetHandleVerifier [0x00F7456C+2775612]
	GetHandleVerifier [0x00D651E0+616112]
	(No symbol) [0x00C75F8C]
	(No symbol) [0x00C72328]
	(No symbol) [0x00C7240B]
	(No symbol) [0x00C64FF7]
	BaseThreadInitThunk [0x75845D49+25]
	RtlInitializeExceptionChain [0x77E1D6DB+107]
	RtlGetAppContainerNamedObjectPath [0x77E1D661+561]


# Tìm kiếm tất cả các link title công việc

lấy trường data-search--job-selection-job-url-value ở trong các thẻ .job-card ở trong thẻ .card-jobs-list    

In [6]:
def find_url_title_job(driver, search_keyword, num_pages):
    job_links = []
    base_url = "https://itviec.com/it-jobs"
    for page in range(1, num_pages + 1):
        search_url = f"{base_url}?search={search_keyword}&page={page}"
        driver.get(search_url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        job_cards = soup.find_all('a', class_='job-link')

        for job_card in job_cards:
            job_link = "https://itviec.com" + job_card['href']
            job_links.append(job_link)

        print(f"Đã thu thập {len(job_links)} liên kết công việc từ trang {page}")

    return job_links